In [33]:
# nuclio: ignore
import nuclio

In [34]:
%nuclio config kind = "job"
%nuclio config spec.image = "python:3.6-jessie"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'python:3.6-jessie'


In [35]:
%%nuclio cmd -c 
pip install requests

In [36]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [37]:
import os
import json
import requests
from mlrun.execution import MLClientCtx
from typing import List

In [38]:
def slack_notify(
    context: MLClientCtx,
    webhook_url: str = "URL",
    slack_blocks:  List[str] = [],
    notification_text: str = "Notification"
) -> None:
    """Summarize a table
    :param context:         the function context
    :param webhook_url:     Slack incoming webhook URL. Please read: https://api.slack.com/messaging/webhooks
    :param notification_text:            Notification text
    :param slack_blocks:          Message blocks list. NOT IMPLEMENTED YET
    """
    
    data = {
        'text': notification_text
    }
    print("====",webhook_url)
    response = requests.post(webhook_url, data=json.dumps(
        data), headers={'Content-Type': 'application/json'})

    print('Response: ' + str(response.text))
    print('Response code: ' + str(response.status_code))

In [39]:
# nuclio: end-code

### mlconfig

In [40]:
from mlrun import mlconf
import os

mlconf.dbpath = 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [ ]:
from mlrun import code_to_function

# create job function object from notebook code
fn = code_to_function("slack_notify")
# add metadata (for templates and reuse)
fn.spec.default_handler = "slack_notify"
fn.spec.description = "Send Slack notification"
fn.metadata.categories = ["ops"]
fn.metadata.labels = {"author": "mdl"}
fn.export("function.yaml")

## tests

In [ ]:
from mlrun import import_function
func = import_function("hub://slack_notify")

In [ ]:
from mlrun import NewTask, run_local


#Slack incoming webhook URL. Please read: https://api.slack.com/messaging/webhooks
task_params = {
    "webhook_url" : "https://hooks.slack.com/services/xxxxxxxx/xxxxxxxxx/xxxxxxxxxxxxxx",
    "notification_text" : "Test Notification"
}

In [ ]:
task = NewTask(
    name="tasks slack notify", 
    params = task_params,
    handler=slack_notify)

### run local where artifact path is fixed 

In [ ]:
run = run_local(task, artifact_path=mlconf.artifact_path)

### run remote where artifact path includes the run id

In [ ]:
func.deploy()

In [ ]:
func.run(task, params=task_params,  workdir=mlconf.artifact_path)

In [42]:
func.doc()

function: slack-notify
Send Slack notification
default handler: slack_notify
entry points:
  slack_notify: Summarize a table
    context(MLClientCtx)  - the function context
    webhook_url(str)  - Slack incoming webhook URL. Please read: https://api.slack.com/messaging/webhooks, default=URL
    slack_blocks(List[str])  - Message blocks list. NOT IMPLEMENTED YET
    notification_text(str)  - Notification text, default=Notification
